In [41]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split


digit = load_digits()

data_x = digit.data
data_y = digit.target


x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=42)


In [42]:
# 查看一下
x_train.shape # 64 -> 8 * 8

(1437, 64)

In [43]:
# 导入想换类库
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout,Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [44]:
num_classes = 10

# input image dimensions
img_rows, img_cols = 8, 8

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0],1,img_rows,img_cols)
    x_test = x_test.reshape(x_test.shape[0],1,img_rows,img_cols)
    input_shape = (1,img_rows,img_cols) # 1 就是通道数， 3 就是RGB， 1 = 黑白的/灰度图
    print (input_shape)
else:
    x_train = x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
    x_test = x_test.reshape(x_test.shape[0],img_rows,img_cols,1)
    input_shape = (img_rows,img_cols,1)
    print (input_shape)

(8, 8, 1)


In [45]:
# 0~1 float

x_train = x_train.astype("float32")
x_test  = x_test.astype("float32")
x_train /= 255
x_test  /=255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (1437, 8, 8, 1)
1437 train samples
360 test samples


In [46]:
y_train = keras.utils.to_categorical(y_train,num_classes) # one-hot vector , 因为softmax要用
y_test = keras.utils.to_categorical(y_test,num_classes)

model =Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu',
          input_shape=input_shape)) #32C3

model.add(Conv2D(64, kernel_size=(3,3), activation='relu')) #64C3

model.add(MaxPooling2D(pool_size=(2,2))) #MP2
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128,activation='relu')) # Dense前都要接Flatten , 6C5-MP2-16C5-MP2-120C1    (FLATTEN)   -84N-10N

model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [47]:
batch_size = 128
epochs = 12
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

Train on 1437 samples, validate on 360 samples
Epoch 1/12
1437/1437 [==============================] - 1s - loss: 2.3012 - acc: 0.1294 - val_loss: 2.2997 - val_acc: 0.1028
Epoch 2/12
1437/1437 [==============================] - 0s - loss: 2.2986 - acc: 0.1594 - val_loss: 2.2970 - val_acc: 0.1694
Epoch 3/12
1437/1437 [==============================] - 0s - loss: 2.2944 - acc: 0.1837 - val_loss: 2.2934 - val_acc: 0.2000
Epoch 4/12
1437/1437 [==============================] - 0s - loss: 2.2908 - acc: 0.2088 - val_loss: 2.2883 - val_acc: 0.2222
Epoch 5/12
1437/1437 [==============================] - 0s - loss: 2.2829 - acc: 0.2331 - val_loss: 2.2792 - val_acc: 0.2250
Epoch 6/12
1437/1437 [==============================] - 0s - loss: 2.2744 - acc: 0.2387 - val_loss: 2.2644 - val_acc: 0.4139
Epoch 7/12
1437/1437 [==============================] - 0s - loss: 2.2531 - acc: 0.2930 - val_loss: 2.2386 - val_acc: 0.5611
Epoch 8/12
1437/1437 [==============================] - 0s - loss: 2.2204 - ac

In [48]:
import numpy as np

y_pred = np.argmax(model.predict(x_train),axis=1) # 把one Hot vector 转换回到数字